<a href="https://colab.research.google.com/github/kalai2315/Langchain_for_AgentiAI/blob/main/Customer_Support_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mini Project 1: Building a Customer Support Agent

In [ ]:
%pip install -U --quiet langchain langchain-google-genai langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 806.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [ ]:
import os
from getpass import getpass

os.environ["GOOGLE_API_KEY"] = getpass("Enter your Google API Key: ")

Enter your Google API Key: ··········


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model = "gemini-1.5-flash-latest")

### **Linking multiple chains** sequentially in LangChain means executing multiple Chain objects one after the other, where the output of one chain becomes the input of the next.

In [ ]:
it_support_queue = [
    "I can't access my email. It keeps showing an error message. Please help.",
    "Tengo problemas con la VPN. No puedo conectarme a la red de la empresa. ¿Pueden ayudarme, por favor?",
    "Mon imprimante ne répond pas et n'imprime plus. J'ai besoin d'aide pour la réparer.",
    "我无法访问公司的网站。每次都显示错误信息。请帮忙解决。"
]
it_support_queue

["I can't access my email. It keeps showing an error message. Please help.",
 'Tengo problemas con la VPN. No puedo conectarme a la red de la empresa. ¿Pueden ayudarme, por favor?',
 "Mon imprimante ne répond pas et n'imprime plus. J'ai besoin d'aide pour la réparer.",
 '我无法访问公司的网站。每次都显示错误信息。请帮忙解决。']

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
from langchain_core.output_parsers import StrOutputParser

In [ ]:
#chain 1: Detect customer message language

prompt1 = """
          Act as a customer support agent.
          For the customer support message delimited below by triple backticks.
          Output the language of the message by one word. eg: spanish

          customer support message: ```{orig_message}```
          """
prompt_template1 = ChatPromptTemplate.from_template(prompt1)
llm_chain1 = (prompt_template1 | llm | StrOutputParser())

In [ ]:
prompt_template1 = ChatPromptTemplate.from_template(prompt1)

In [ ]:
llm_chain1 = (prompt_template1 | llm | StrOutputParser())

In [ ]:
it_support_queue[1]

'Tengo problemas con la VPN. No puedo conectarme a la red de la empresa. ¿Pueden ayudarme, por favor?'

In [ ]:
{"orig_message": it_support_queue[1]}

{'orig_message': 'Tengo problemas con la VPN. No puedo conectarme a la red de la empresa. ¿Pueden ayudarme, por favor?'}

In [ ]:
llm_chain1.invoke({"orig_message": it_support_queue[1]})

'Spanish'

In [ ]:
from langchain.schema.runnable import RunnablePassthrough

RunnablePassthrough.assign(language=llm_chain1).invoke({"orig_message": it_support_queue[1]})

{'orig_message': 'Tengo problemas con la VPN. No puedo conectarme a la red de la empresa. ¿Pueden ayudarme, por favor?',
 'language': 'Spanish'}

In [ ]:
#chain2: translate the customer message to English

prompt2 = """
          Act as a customer support agent.
          For the customer support message and customer support message language delimited below by triple backticks.
          Translate the customer support message from customer support message language to English.
          if customer message language is not in English
          else return back the customer support message as it is.

          customer support message: ```{orig_message}```
          customer support message language: ```{orig_language}```
          """
prompt_template2 = ChatPromptTemplate.from_template(prompt2)

llm_chain2 = (prompt_template2 | llm | StrOutputParser())


In [ ]:
tran_message = llm_chain2.invoke({
    "orig_message": it_support_queue[1],
    "orig_language": 'Spanish'
})

In [ ]:
tran_message

"I'm having problems with the VPN. I can't connect to the company network. Can you please help me?"

In [ ]:
#chain3: Generate a resolution response in English

prompt3 = """
          Act as a customer support agent.
          For the customer support message delimited below by triple backticks.
          Generate a resolution response in English.

          customer support message: ```{tran_message}```
          """
prompt_template3 = ChatPromptTemplate.from_template(prompt3)

llm_chain3 = (prompt_template3 | llm | StrOutputParser())


In [ ]:
tran_response = llm_chain3.invoke({"tran_message": tran_message})
print(tran_response)

Thank you for contacting support. I understand you're having trouble connecting to the company network via VPN.  To help me troubleshoot this issue, could you please provide me with the following information:

* **What operating system are you using?** (e.g., Windows 10, macOS Monterey, iOS 16, Android 13)
* **What VPN client are you using?** (e.g., Cisco AnyConnect, OpenVPN, etc.)
* **What error message, if any, are you receiving?** Please provide the exact wording.
* **Have you tried restarting your computer and/or your router?**
* **Are you able to connect to the internet outside of the VPN?**

Once I have this information, I can better assist you in resolving the connection issue.  In the meantime, you may also want to check your company's internal network documentation or helpdesk for any known outages or issues.


In [ ]:
#chain4: Translate the resolution response from English to the Customer's language

prompt4 = """
          Act as a customer support agent.
          For the customer resolution response and target language delimited below by triple backticks.
          Translate the resolution response from English to the target language.
          if customer message language is not in English
          else return back the customer support message as it is.

          customer resolution response: ```{tran_response}```
          targetlanguage: ```{orig_language}```
          """
prompt_template4 = ChatPromptTemplate.from_template(prompt4)

llm_chain4 = (prompt_template4 | llm | StrOutputParser())

In [ ]:
# Step 4: Translate response to customer's language
final_response = llm_chain4.invoke({
    "tran_response": tran_response,
    "orig_language": 'Spanish'
})

In [ ]:
print(final_response)

```
Gracias por contactar con el soporte técnico. Entiendo que tiene problemas para conectarse a la red de la empresa a través de VPN. Para ayudarme a solucionar este problema, ¿podría proporcionarme la siguiente información?

* **¿Qué sistema operativo está utilizando?** (p. ej., Windows 10, macOS Monterey, iOS 16, Android 13)
* **¿Qué cliente VPN está utilizando?** (p. ej., Cisco AnyConnect, OpenVPN, etc.)
* **¿Qué mensaje de error, si hay alguno, está recibiendo?** Por favor, proporcione la redacción exacta.
* **¿Ha intentado reiniciar su ordenador y/o su router?**
* **¿Puede conectarse a internet fuera de la VPN?**

Una vez que tenga esta información, podré ayudarle mejor a resolver el problema de conexión. Mientras tanto, también puede consultar la documentación de la red interna de su empresa o el servicio de asistencia técnica para ver si hay alguna interrupción o problema conocido.
```


In [ ]:
# final_chain: sequentially processes the customer message
final_chain = (
    RunnablePassthrough.assign(orig_language=llm_chain1)  # Step 1: Detect language
    .assign(tran_message=llm_chain2)                      # Step 2: Translate message to English
    .assign(tran_response=llm_chain3)                     # Step 3: Generate resolution in English
    .assign(final_response=llm_chain4)                    # Step 4: Translate resolution to original language
)

In [ ]:
{"orig_message": it_support_queue[1]}

{'orig_message': 'Tengo problemas con la VPN. No puedo conectarme a la red de la empresa. ¿Pueden ayudarme, por favor?'}

In [ ]:
response = final_chain.invoke({"orig_message": it_support_queue[1]})
response

{'orig_message': 'Tengo problemas con la VPN. No puedo conectarme a la red de la empresa. ¿Pueden ayudarme, por favor?',
 'orig_language': 'Spanish',
 'tran_message': "I'm having problems with the VPN. I can't connect to the company network. Can you please help me?",
 'tran_response': "Subject: Re: VPN Connection Issue\n\nHi [Customer Name],\n\nThanks for contacting us about your VPN connection issue.  I understand you're unable to connect to the company network.  To help me troubleshoot this, could you please provide me with the following information:\n\n* **Your operating system:** (e.g., Windows 10, macOS Monterey, iOS 16)\n* **Your device:** (e.g., Laptop, Desktop, Mobile Phone)\n* **The exact error message you are receiving:** (If any, please copy and paste the message)\n* **Have you tried restarting your computer/device?**\n* **Have you tried restarting your router/modem?**\n* **What VPN client are you using?** (e.g., Cisco AnyConnect, OpenVPN)\n* **Have you checked your company'

In [ ]:
it_support_queue

["I can't access my email. It keeps showing an error message. Please help.",
 'Tengo problemas con la VPN. No puedo conectarme a la red de la empresa. ¿Pueden ayudarme, por favor?',
 "Mon imprimante ne répond pas et n'imprime plus. J'ai besoin d'aide pour la réparer.",
 '我无法访问公司的网站。每次都显示错误信息。请帮忙解决。']

In [ ]:
it_support_queue_formatted = [{"orig_message": msg} for msg in it_support_queue]
it_support_queue_formatted

[{'orig_message': "I can't access my email. It keeps showing an error message. Please help."},
 {'orig_message': 'Tengo problemas con la VPN. No puedo conectarme a la red de la empresa. ¿Pueden ayudarme, por favor?'},
 {'orig_message': "Mon imprimante ne répond pas et n'imprime plus. J'ai besoin d'aide pour la réparer."},
 {'orig_message': '我无法访问公司的网站。每次都显示错误信息。请帮忙解决。'}]

In [ ]:
response = final_chain.map().invoke(it_support_queue_formatted)
response

[{'orig_message': "I can't access my email. It keeps showing an error message. Please help.",
  'orig_language': 'English',
  'tran_message': "I can't access my email. It keeps showing an error message. Please help.",
  'tran_response': "I understand you're having trouble accessing your email and are receiving an error message.  To help me assist you better, could you please provide me with the following information:\n\n* **What is the exact error message you are seeing?** (Please copy and paste the message if possible).\n* **What email provider are you using?** (e.g., Gmail, Yahoo, Outlook, etc.)\n* **What device are you using to access your email?** (e.g., computer, phone, tablet)\n* **What operating system are you using?** (e.g., Windows 10, iOS 16, Android 13)\n* **Have you tried restarting your device?**\n* **Have you checked your internet connection?**\n\nOnce I have this information, I can provide more specific troubleshooting steps.  We'll get this resolved for you!",
  'final_

In [ ]:
import pandas as pd
pd.DataFrame(response)

,orig_message,orig_language,tran_message,tran_response,final_response
0,I can't access my email. It keeps showing an e...,English,I can't access my email. It keeps showing an e...,I understand you're having trouble accessing y...,I understand you're having trouble accessing y...
1,Tengo problemas con la VPN. No puedo conectarm...,Spanish,I'm having problems with the VPN. I can't conn...,Thank you for contacting support. I understand...,```\nGracias por contactar con el soporte. Ent...
2,Mon imprimante ne répond pas et n'imprime plus...,French,My printer is not responding and no longer pri...,I'm sorry to hear you're having trouble with y...,```\nJe suis désolé d'apprendre que vous renco...
3,我无法访问公司的网站。每次都显示错误信息。请帮忙解决。,Chinese,I cannot access the company website. I get an...,I understand you're having trouble accessing o...,```\n我了解您在访问我们的网站时遇到问题并收到错误消息。为了帮助您解决这个问题，请您提供...
